<a href="https://colab.research.google.com/github/ModernOctave/Thermal-Image-Human-Detection/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pull Code

In [ ]:
!git clone -b v2 https://github.com/ModernOctave/Thermal-Image-Human-Detection.git
%cd Thermal-Image-Human-Detection

# Download Raw Dataset

In [2]:
!wget -v https://adas-dataset-v2.flirconservator.com/dataset/full/FLIR_ADAS_v2.zip
!unzip -q FLIR_ADAS_v2.zip -d FLIR_ADAS_v2

--2022-07-15 09:50:37--  https://adas-dataset-v2.flirconservator.com/dataset/full/FLIR_ADAS_v2.zip
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving adas-dataset-v2.flirconservator.com (adas-dataset-v2.flirconservator.com)... 54.182.0.78, 54.182.0.101, 54.182.0.56, ...
Connecting to adas-dataset-v2.flirconservator.com (adas-dataset-v2.flirconservator.com)|54.182.0.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11865443849 (11G) [application/zip]
Saving to: ‘FLIR_ADAS_v2.zip’

FLIR_ADAS_v2.zip    100%[===================>]  11.05G  20.3MB/s    in 9m 22s  

2022-07-15 10:00:00 (20.2 MB/s) - ‘FLIR_ADAS_v2.zip’ saved [11865443849/11865443849]



# Install Dependancies


In [1]:
import tensorflow as tf

2022-07-19 13:32:54.686167: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-07-19 13:32:54.727714: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-19 13:32:54.727733: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Run the following if using colab,

In [ ]:
# Import Tensorflow 2.0
%tensorflow_version 2.x

# Check that we are using a GPU, if not switch runtimes
#   using Runtime > Change Runtime Type > GPU
assert len(tf.config.list_physical_devices('GPU')) > 0

# Create datasets

In [2]:
from preprocessing import label_dataset, split_dataset, DataGen

labeled_dataset = label_dataset(dataset_path="FLIR_ADAS_v2/images_thermal_train/")
train_dataset, validation_dataset = split_dataset(labeled_dataset, 0.8, shuffle=True)

train_gen = DataGen(train_dataset, batch_size=32)
test_gen = DataGen(test_dataset, batch_size=32)

loading annotations into memory...
Done (t=1.11s)
creating index...
index created!
labeling the dataset...


100%|██████████| 10742/10742 [00:00<00:00, 148386.59it/s]


# Model


## Create the model

In [6]:
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2S
from tensorflow.keras.metrics import BinaryAccuracy, AUC, FalsePositives, FalseNegatives

model = tf.keras.Sequential([
    tf.keras.layers.Resizing(384, 384, input_shape=(512, 640, 3)),
    EfficientNetV2S(include_top=False),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.Adam(),
             loss=tf.keras.losses.BinaryCrossentropy(),
             metrics=[BinaryAccuracy(), AUC(), FalsePositives(), FalseNegatives()])

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing_1 (Resizing)       (None, 384, 384, 3)       0         
                                                                 
 efficientnetv2-s (Functiona  (None, None, None, 1280)  20331360 
 l)                                                              
                                                                 
 global_average_pooling2d_1   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_1 (Dense)             (None, 1)                 1281      
                                                                 
Total params: 20,332,641
Trainable params: 20,178,769
Non-trainable params: 153,872
_________________________________________________________________
None


# Load or Save the Model

In [ ]:
model.save('model')

In [ ]:
model = tf.keras.models.load_model('model')

## Train the model

In [11]:
from tensorflow.keras.callbacks import EarlyStopping

callbacks = [EarlyStopping(patience=2, restore_best_weights=True)]

model.fit(train_gen, epochs=1000, validation_data=test_gen, callbacks=callbacks)

Epoch 1/10
  1/269 [..............................] - ETA: 3:22:38 - loss: 0.7309 - binary_accuracy: 0.5000

## Test the model


In [ ]:
loss, accuracy = cnn_model.evaluate(test_images, test_labels, batch_size=1)